<a href="https://colab.research.google.com/github/vishwvir-singh/Text-completion-with-GPT-2/blob/main/Text_completion_with_GPT-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/Colab_Notebooks_data/Text-completion-with-GPT-2

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Colab_Notebooks_data/Text-completion-with-GPT-2


In [20]:
#Clone GPT-2
!git clone https://github.com/openai/gpt-2.git
%cd gpt-2/

fatal: destination path 'gpt-2' already exists and is not an empty directory.
/gdrive/My Drive/Colab_Notebooks_data/Text-completion-with-GPT-2/gpt-2


In [23]:
%ll ./

total 35
-rw------- 1 root   551 Jun  5 18:13 CONTRIBUTORS.md
-rw------- 1 root  2188 Jun  5 18:13 DEVELOPERS.md
-rw------- 1 root   279 Jun  5 18:13 Dockerfile.cpu
-rw------- 1 root   548 Jun  5 18:13 Dockerfile.gpu
-rw------- 1 root 14754 Jun  5 18:13 domains.txt
-rw------- 1 root  1075 Jun  5 18:13 download_model.py
-rw------- 1 root  1403 Jun  5 18:13 LICENSE
-rw------- 1 root  4989 Jun  5 18:13 model_card.md
-rw------- 1 root  2827 Jun  5 18:13 README.md
-rw------- 1 root    58 Jun  5 18:13 requirements.txt
drwx------ 2 root  4096 Jun  5 18:13 src/


In [37]:
%ll src

total 21
-rw------- 1 root 4242 Jun  5 18:13 encoder.py
-rw------- 1 root 2870 Jun  5 18:13 generate_unconditional_samples.py
-rw------- 1 root 3412 Jun  5 18:13 interactive_conditional_samples.py
-rw------- 1 root 6503 Jun  5 18:13 model.py
-rw------- 1 root 3166 Jun  5 18:13 sample.py


In [25]:
!pip install -qr requirements.txt

In [27]:
#GPT2 runs on tf 1.x. So lets change the tf version
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.2


In [35]:
import os
if not os.path.exists('models/345M'):
  !python download_model.py '345M'
%ll models

total 4
drwx------ 2 root 4096 Jun  5 18:40 345M/


In [36]:
!export PYTHONIOENCODING=UTF-8

In [38]:
from src import model

In [39]:
help(model)

Help on module src.model in src:

NAME
    src.model

FUNCTIONS
    attention_mask(nd, ns, *, dtype)
        1's in the lower triangle, counting from the lower right corner.
        
        Same as tf.matrix_band_part(tf.ones([nd, ns]), -1, ns-nd), but doesn't produce garbage on TPUs.
    
    attn(x, scope, n_state, *, past, hparams)
    
    block(x, scope, *, past, hparams)
    
    conv1d(x, scope, nf, *, w_init_stdev=0.02)
    
    default_hparams()
    
    expand_tile(value, size)
        Add a new axis of given size.
    
    gelu(x)
    
    merge_states(x)
        Smash the last two dimensions of x into a single dimension.
    
    mlp(x, scope, n_state, *, hparams)
    
    model(hparams, X, past=None, scope='model', reuse=False)
    
    norm(x, scope, *, axis=-1, epsilon=1e-05)
        Normalize to mean = 0, std = 1, then do a diagonal affine transform.
    
    past_shape(*, hparams, batch_size=None, sequence=None)
    
    positions_for(tokens, past_length)
    
    sha